In [1]:
# importation des packages

import pandas as pd
import numpy as np
import seaborn as sns
import plotly as plt
import plotly.express as px
import plotly.offline as pyo
from dash.dependencies import Output, Input
import json

In [2]:
# Suppression des warnings

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')
print("No Warning Shown")

No Warning Shown


In [3]:
# importation Dataset

df = pd.read_csv("Températures_par_pays_Vertical.csv")
df.head()

,Year,Month,Anomaly,Country
0,1874,12,-0.330,South Georgia And The South Sandwich Isla
1,1875,1,-1.135,South Georgia And The South Sandwich Isla
2,1875,2,-1.087,South Georgia And The South Sandwich Isla
3,1875,3,-0.716,South Georgia And The South Sandwich Isla
4,1875,4,-0.892,South Georgia And The South Sandwich Isla


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544390 entries, 0 to 544389
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Year     544390 non-null  int64  
 1   Month    544390 non-null  int64  
 2   Anomaly  518063 non-null  float64
 3   Country  544390 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 16.6+ MB


In [4]:
# Calcul de température moyenne par pays
Temp_country_mean = df.groupby(['Country', 'Year']).mean().reset_index()
Temp_country_mean

,Country,Year,Month,Anomaly
0,Afghanistan,1848,8.5,-0.810625
1,Afghanistan,1849,6.5,-0.698833
2,Afghanistan,1850,6.5,-0.743667
3,Afghanistan,1851,6.5,-0.478333
4,Afghanistan,1852,6.5,NaN
...,...,...,...,...
45431,Zimbabwe,2016,6.5,1.578333
45432,Zimbabwe,2017,6.5,0.500083
45433,Zimbabwe,2018,6.5,1.050000
45434,Zimbabwe,2019,6.5,1.608167


In [5]:
# Selection de l'année à partir de 1880

Temp_country_mean = Temp_country_mean[Temp_country_mean['Year'] >= 1880]

In [6]:
# creation d'une fonction pour créer une colonne classe en fonction des variations
def map_anomaly(x):
    if x < 1:
        return 'moins de 1°'
    elif x < 2:
        return '1° à 2°'
    elif x < 3:
        return '2° à 3°'
    else:
        return 'plus de 3°'

# use the apply() function to create a new column based on the values in the 'Anomaly' column
Temp_country_mean['Classe'] = Temp_country_mean['Anomaly'].apply(map_anomaly)


In [7]:
# Pays avec les anomalies les plus importantes
top_15 = Temp_country_mean.groupby('Country').sum().sort_values('Anomaly', ascending=False)[:15].reset_index()['Country']
top_15

0                     Russia
1                    Estonia
2                     Latvia
3                    Finland
4     Svalbard And Jan Mayen
5                  Lithuania
6                     Sweden
7                    Belarus
8                     Norway
9                    Tunisia
10                Mauritania
11                     Malta
12             Liechtenstein
13            Western Sahara
14                    Poland
Name: Country, dtype: object

In [8]:
# Pays avec les anomalies les moins importantes
bottom_15 = Temp_country_mean.groupby('Country').sum().sort_values('Anomaly', ascending=True)[:15].reset_index()['Country']
bottom_15

0             Saint Pierre And Miquelon
1                              CuraçAo
2                             Hong Kong
3                                 Macau
4                          Baker Island
5     Bonaire, Saint Eustatius And Saba
6                                 Aruba
7                           New Zealand
8                                 Japan
9                            Martinique
10                                 Niue
11                               Taiwan
12                            Venezuela
13                            Greenland
14                           Guadeloupe
Name: Country, dtype: object

In [11]:
# Visualisation des anomalies de température pour les pays les plus impactés
Temp_country_mean_top = Temp_country_mean[Temp_country_mean['Country'].isin(top_15)]
fig = px.bar(Temp_country_mean_top, x='Anomaly', y='Country', animation_frame='Year', hover_name='Anomaly',
      range_x=[-3,5],color='Country', labels={
                     "Anomaly": "Anomalie",
                     "Country": "Pays"}, title = "Anomalies de température pour les 15 pays les plus impactés",width=1000, height=500)


pyo.plot(fig, filename="Anomalies de température pour les 15 pays les plus impactés.html")

'Anomalies de température pour les 15 pays les plus impactés.html'

In [ ]:
# Visualisation des anomalies de température pour les pays les moins impactés
Temp_country_mean_bottom = Temp_country_mean[Temp_country_mean['Country'].isin(bottom_15)]
fig = px.bar(Temp_country_mean_bottom,x='Anomaly', y='Country', animation_frame='Year', hover_name='Anomaly',
      range_x=[-3,5], color='Country', labels={
                     "Anomaly": "Anomalie",
                     "Country": "Pays"}, title = "Anomalies de température pour les 15 pays les moins impactés", width=1000, height=500)

pyo.plot(fig, filename="Anomalies de température pour les 15 pays les moins impactés.html")

In [ ]:
# Visualisation des anomalies de température pour les pays les plus impactés

fig = px.line(Temp_country_mean_top, x="Year", y="Anomaly", color="Country", title="Anomalies de température pour les 15 pays les plus impactés")
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
            )
)

pyo.plot(fig, filename="Courbes des anomalies de température pour les 15 pays les plus impactés.html")

In [9]:
# remplacement de noms de pays en corrélation avec les nom s dans notre fichier geaojson

Temp_country_mean = Temp_country_mean.replace(to_replace=["Dominican", "Burma", "Bosnia And Herzegovina", "Czech Republic", 'United States', "Ivorycoast", "Congo (Democratic Republic Of The)", 'Tanzania'], value=["Dominican Republic", "Myanmar", "Bosnia and Herzegovina", "Czechia", "United States of America", "Ivory Coast", "Democratic Republic of the Congo", "United Republic of Tanzania"])


In [10]:
# Importation du fichier geojson

with open('world.geo.json') as response:
 world = json.load(response) # Javascrip object notation 


In [11]:
state_id_map = {}
for feature in world ['features']:
 feature['id'] = feature['properties']['admin']
 state_id_map[feature['properties']['adm0_a3']] = feature['id']

In [18]:
# Création d'un graphe interactif avec les anomalies de température par pays en fonction de l'année

fig = px.choropleth(
 Temp_country_mean, #database
 locations = 'Country', #define the limits on the map/geography
 geojson = world, #shape information
 color = "Anomaly", #defining the color of the scale through the database
 color_continuous_scale=px.colors.sequential.Rainbow,
 hover_name = "Country", #the information in the box
 hover_data =["Country"],
 title = "Anomalies de température par pays", #title of the map
 animation_frame = "Year" #creating the application based on the year
)
fig.update_geos(fitbounds = "locations", visible = True)
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
            )
)

pyo.plot(fig, filename="Anomalies de température par pays.html")

'Anomalies de température par pays.html'

In [20]:
# Create the interactive map using Plotly
fig = px.choropleth(Temp_country_mean,
                    locations="Country",
                    locationmode="country names",
                    color="Classe",
                    animation_frame="Year",
                    range_color=[0, 4],  # Set the color scale range to match the number of classes
                    color_discrete_sequence=px.colors.qualitative.Pastel1,
                    title="Anomalies de températures par pays et par catégories",
                    labels={"Classe": "Catégories de température"})

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
            )
)


pyo.plot(fig, filename="Anomalies de température par pays et par catégories.html")


'Anomalies de température par pays et par catégories.html'

In [12]:
# création d'un dataframe avec les moyenne par année

Temp_country_mean_2 = df.groupby(['Country', 'Year']).mean().reset_index()

In [14]:
# Création d'un graphe interactif des températures avec selecteur multi-pays

import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash

# Create a Dash app
app = JupyterDash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id="country-selector",
        options=[{"label": country, "value": country} for country in Temp_country_mean_2["Country"].unique()],
        value=["Afghanistan"],
        multi=True
    ),
    dcc.Graph(id="line-plot")
])


# Define the callback function that updates the plot when the dropdown selection changes
@app.callback(
    dash.dependencies.Output("line-plot", "figure"),
    [dash.dependencies.Input("country-selector", "value")]
)
def update_plot(selected_countries):
    # Filter the DataFrame to only include the selected countries
    filtered_df = Temp_country_mean_2[Temp_country_mean_2["Country"].isin(selected_countries)]
    
    # Create the line plot using Plotly
    fig = px.line(filtered_df, x="Year", y="Anomaly", color="Country", title="Anomalies de températures par pays")
    
    # Set the width and height of the plot
    fig.update_layout(width=1000, height=600)
    
    # Return the plot
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8056)


Dash app running on http://127.0.0.1:8056/


In [15]:
Temp_country_mean

,Country,Year,Month,Anomaly,Classe
32,Afghanistan,1880,6.5,0.163833,moins de 1°
33,Afghanistan,1881,6.5,0.297167,moins de 1°
34,Afghanistan,1882,6.5,-0.568750,moins de 1°
35,Afghanistan,1883,6.5,-0.359333,moins de 1°
36,Afghanistan,1884,6.5,-0.574333,moins de 1°
...,...,...,...,...,...
45431,Zimbabwe,2016,6.5,1.578333,1° à 2°
45432,Zimbabwe,2017,6.5,0.500083,moins de 1°
45433,Zimbabwe,2018,6.5,1.050000,1° à 2°
45434,Zimbabwe,2019,6.5,1.608167,1° à 2°


In [16]:
df_co2 = pd.read_csv('co2_par_pays.csv')
df_co2

,country,co2,year
0,Afghanistan,NaN,1880
1,Afghanistan,NaN,1881
2,Afghanistan,NaN,1882
3,Afghanistan,NaN,1883
4,Afghanistan,NaN,1884
...,...,...,...
35373,Zimbabwe,9.596,2017
35374,Zimbabwe,11.795,2018
35375,Zimbabwe,11.115,2019
35376,Zimbabwe,10.608,2020


In [17]:
df_co2 = df_co2.rename({'country':'Country', 'year':'Year'}, axis=1)
df_co2

,Country,co2,Year
0,Afghanistan,NaN,1880
1,Afghanistan,NaN,1881
2,Afghanistan,NaN,1882
3,Afghanistan,NaN,1883
4,Afghanistan,NaN,1884
...,...,...,...
35373,Zimbabwe,9.596,2017
35374,Zimbabwe,11.795,2018
35375,Zimbabwe,11.115,2019
35376,Zimbabwe,10.608,2020


In [18]:
df_temp_co2 = Temp_country_mean.merge(right = df_co2, on = ['Year', 'Country'], how = 'outer')
df_temp_co2

,Country,Year,Month,Anomaly,Classe,co2
0,Afghanistan,1880,6.5,0.163833,moins de 1°,NaN
1,Afghanistan,1881,6.5,0.297167,moins de 1°,NaN
2,Afghanistan,1882,6.5,-0.568750,moins de 1°,NaN
3,Afghanistan,1883,6.5,-0.359333,moins de 1°,NaN
4,Afghanistan,1884,6.5,-0.574333,moins de 1°,NaN
...,...,...,...,...,...,...
41717,Zimbabwe,1881,NaN,NaN,NaN,NaN
41718,Zimbabwe,1882,NaN,NaN,NaN,NaN
41719,Zimbabwe,1883,NaN,NaN,NaN,NaN
41720,Zimbabwe,1884,NaN,NaN,NaN,NaN


In [21]:
# Création d'un graphe interactif des températures et CO2 avec selecteur de pays


import dash
import dash_core_components as dcc
import dash_html_components as html


# Get a list of unique countries in the dataframe
countries = df_temp_co2['Country'].unique().tolist()

# Create a Dash app
app = JupyterDash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Anomalies de températures and Emissions de CO2"),
    html.Div([
        html.Div([
            # Create a dropdown menu to select the country
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': c, 'value': c} for c in countries],
                value=countries[0]
            ),
            # Create a div to hold the Anomaly plot
            dcc.Graph(id='anomaly-plot')
        ], className="six columns"),
        html.Div([
            # Create a div to hold the CO2 plot
            dcc.Graph(id='co2-plot')
        ], className="six columns")
    ], className="row")
])

# Define the app callback to update the Anomaly plot when a new country is selected
@app.callback(
    dash.dependencies.Output('anomaly-plot', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_anomaly_plot(country):
    fig = px.line(df_temp_co2[df_temp_co2['Country'] == country], x='Year', y='Anomaly')
    fig.update_layout(title=f"{country} - Temperature Anomaly")
    
    # Set the width and height of the plot
    fig.update_layout(width=1000, height=400)
    
    return fig

# Define the app callback to update the CO2 plot when a new country is selected
@app.callback(
    dash.dependencies.Output('co2-plot', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_co2_plot(country):
    fig = px.line(df_temp_co2[df_temp_co2['Country'] == country], x='Year', y='co2')
    fig.update_layout(title=f"{country} - CO2 Emissions")
    
    # Set the width and height of the plot
    fig.update_layout(width=1000, height=400)
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8053)


Dash app running on http://127.0.0.1:8053/


In [20]:
# Création d'un graphe interactif des températures et C02 avec selecteur multi-pays


import dash
import dash_core_components as dcc
import dash_html_components as html

# Get a list of unique countries in the dataframe
countries = df_temp_co2['Country'].unique().tolist()

# Create a Dash app
app = JupyterDash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Anomalies de températures and Emissions de CO2 par pays"),
    html.Div([
        html.Div([
            # Create a dropdown menu to select the country
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': c, 'value': c} for c in countries],
                value=[countries[0]],
                multi=True
            ),
            # Create a div to hold the Anomaly plot
            dcc.Graph(id='anomaly-plot')
        ], className="six columns"),
        html.Div([
            # Create a div to hold the CO2 plot
            dcc.Graph(id='co2-plot')
        ], className="six columns")
    ], className="row")
])

# Define the app callback to update the Anomaly plot when a new country is selected
@app.callback(
    dash.dependencies.Output('anomaly-plot', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_anomaly_plot(countries):
    # Filter the DataFrame to include only the selected countries
    filtered_df = df_temp_co2[df_temp_co2['Country'].isin(countries)]
    
    # Create the plot using Plotly
    fig = px.line(filtered_df, x='Year', y='Anomaly', color='Country')
    fig.update_layout(title=f"Anomalies de températures")
    
    # Set the width and height of the plot
    fig.update_layout(width=1000, height=400)
    
    return fig

# Define the app callback to update the CO2 plot when a new country is selected
@app.callback(
    dash.dependencies.Output('co2-plot', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_co2_plot(countries):
    # Filter the DataFrame to include only the selected countries
    filtered_df = df_temp_co2[df_temp_co2['Country'].isin(countries)]
    
    # Create the plot using Plotly
    fig = px.line(filtered_df, x='Year', y='co2', color='Country')
    fig.update_layout(title=f"Emissions de CO2 (en millions de tonnes)")
    
    # Set the width and height of the plot
    fig.update_layout(width=1000, height=400)
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8054)

# Export as standalone HTML file
#app.export('Anomalies de températures and Emissions de CO2 par pays.html')    

Dash app running on http://127.0.0.1:8054/
